# Simulate a feedback survey
This notebook provides sample EDSL code for simulating surveys with AI agents and large language models.
In the steps below we use EDSL to prompt LLMs to suggest names for a yoga studio, and then simulate a feedback survey with AI agents representing target customers.

Before running the code below, please ensure that you have [installed the EDSL library](https://docs.expectedparrot.com/en/latest/installation.html) and either [activated remote inference](https://docs.expectedparrot.com/en/latest/remote_inference.html) from your [Coop account](https://docs.expectedparrot.com/en/latest/coop.html) or [stored API keys](https://docs.expectedparrot.com/en/latest/api_keys.html) for the language models that you want to use with EDSL. Please also see our [documentation page](https://docs.expectedparrot.com/) for tips and tutorials on getting started using EDSL.

## Create a question
We start by creating a question to prompt a language model to suggest some names for a yoga studio.
EDSL comes with many common [question types](https://docs.expectedparrot.com/en/latest/questions.html) that we can choose from based on the form of the response that we want to get back from the model - multiple choice, free text, linear scale, etc. 
Here we use `QuestionList` to prompt a model to return a list of items:

In [1]:
from edsl import QuestionList

In [2]:
q = QuestionList(
    question_name = "yoga_studio_name",
    question_text = "What are some creative names for a yoga studio?",
    max_list_items = 10
)

## Select some models to answer the question
EDSL works with many popular [language models that we can select](https://docs.expectedparrot.com/en/latest/language_models.html) to generate responses to questions.

In [3]:
from edsl import ModelList, Model

To see a list of all services:

In [4]:
Model.services()

,Service Name
0,anthropic
1,azure
2,bedrock
3,deep_infra
4,deepseek
5,google
6,groq
7,mistral
8,ollama
9,openai


A list of current available models can be viewed [here](https://www.expectedparrot.com/getting-started/coop-pricing).

To select models to use with a question or survey:

In [5]:
models = ModelList(
    Model(m) for m in ["claude-3-sonnet-20240229", "gpt-4o"]
)

## Run the question 
We administer a question to a language model by calling the `run()` method on it.
This generates a formatted dataset of `Results`:

In [6]:
results = q.by(models).run()

Job UUID,778977cf-4f0f-4e61-bd69-8617650303dd
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/778977cf-4f0f-4e61-bd69-8617650303dd
Exceptions Report URL,None
Results UUID,9e8f4eb2-de88-4a9b-8b97-89d6547c23b8
Results URL,https://www.expectedparrot.com/content/9e8f4eb2-de88-4a9b-8b97-89d6547c23b8


## Inspect the results
EDSL comes with built-in [methods for analyzing results](https://docs.expectedparrot.com/en/latest/results.html).
Here we inspect the responses for each model:

In [7]:
results.select("model", "yoga_studio_name")

,model.model,answer.yoga_studio_name
0,claude-3-sonnet-20240229,"['Zen Haven', 'Mindful Movements', 'Serenity Studio', 'Breathe Bliss', 'Soul Sanctuary', 'Harmony Hive', 'Peaceful Postures', 'Chakra Chalet', 'Nirvana Nook', 'Karma Konnection']"
1,gpt-4o,"['Zen Flow', 'Soul Stretch', 'Harmony Haven', 'Tranquil Twist', 'Serene Space', 'Blissful Balance', 'Mindful Movements', 'Peaceful Pose', 'Inner Light Studio', 'Namaste Nook']"


## Use the responses in new questions
We can format the responses to use them as options to new questions:

In [8]:
claude_names = results.filter("model.model == 'claude-3-sonnet-20240229'").select("yoga_studio_name").to_list()[0]
claude_names

['Zen Haven',
 'Mindful Movements',
 'Serenity Studio',
 'Breathe Bliss',
 'Soul Sanctuary',
 'Harmony Hive',
 'Peaceful Postures',
 'Chakra Chalet',
 'Nirvana Nook',
 'Karma Konnection']

In [9]:
gpt4o_names = results.filter("model.model == 'gpt-4o'").select("yoga_studio_name").to_list()[0]
gpt4o_names

['Zen Flow',
 'Soul Stretch',
 'Harmony Haven',
 'Tranquil Twist',
 'Serene Space',
 'Blissful Balance',
 'Mindful Movements',
 'Peaceful Pose',
 'Inner Light Studio',
 'Namaste Nook']

We can optionally randomize the list of options:

In [10]:
studio_names = list(set(gpt4o_names + claude_names))

In [11]:
import random

random.shuffle(studio_names)

## Constructing a survey
Here we create some new questions in different types and combine them into a survey to administer them together:

In [12]:
from edsl import QuestionMultipleChoice, QuestionCheckBox, Survey

In [13]:
q1 = QuestionMultipleChoice(
    question_name = "favorite",
    question_text = "What is your favorite name for a yoga studio?",
    question_options = studio_names
)

In [14]:
q2 = QuestionCheckBox(
    question_name = "pick",
    question_text = "Pick the 3 best names for a yoga studio.",
    question_options = studio_names,
    min_selections = 3,
    max_selections = 3
)

In [15]:
survey = Survey([q1, q2])

## Designing agents to answer the survey
Next we can design AI agents with relevant traits to answer the questions.
Here we use a model to draft some personas, and then create "agent" objects for them to use with the survey:

In [16]:
q = QuestionList(
    question_name = "personas",
    question_text = "Draft 5 diverse personas for patrons of a yoga studio."
)

In [17]:
personas = q.run().select("personas").to_list()[0]
personas

Job UUID,70bfe9aa-98c4-4de6-9dc9-faaf259efeb3
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/70bfe9aa-98c4-4de6-9dc9-faaf259efeb3
Exceptions Report URL,None
Results UUID,2ab884f4-1ebf-4afb-934c-f0df2a66d700
Results URL,https://www.expectedparrot.com/content/2ab884f4-1ebf-4afb-934c-f0df2a66d700


['A 28-year-old tech professional seeking stress relief and mindfulness after long work hours',
 'A 45-year-old mother of three looking to improve flexibility and find personal time',
 'A 60-year-old retired teacher using yoga to maintain health and social connections',
 'A 22-year-old college athlete incorporating yoga for enhanced performance and injury prevention',
 'A 35-year-old artist exploring yoga for creative inspiration and community involvement']

In [18]:
from edsl import AgentList, Agent

In [19]:
agents = AgentList(
    Agent(traits = {"persona":p}) for p in personas
)

### Run the survey with the agents
Here we add the agents to the survey and run it with the language models we selected above (to compare responses):

In [20]:
results = survey.by(agents).by(models).run()

Job UUID,233ea853-e970-4820-8522-315ebc59afc4
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/233ea853-e970-4820-8522-315ebc59afc4
Exceptions Report URL,None
Results UUID,55fd482d-ddd6-4338-a18e-b0bb420c3084
Results URL,https://www.expectedparrot.com/content/55fd482d-ddd6-4338-a18e-b0bb420c3084


In [21]:
(
    results
    .sort_by("model", "persona")
    .select("model", "persona", "favorite", "pick")
)

,model.model,agent.persona,answer.favorite,answer.pick
0,claude-3-sonnet-20240229,A 22-year-old college athlete incorporating yoga for enhanced performance and injury prevention,Zen Haven,"['Breathe Bliss', 'Blissful Balance', 'Serene Space']"
1,claude-3-sonnet-20240229,A 28-year-old tech professional seeking stress relief and mindfulness after long work hours,Zen Haven,"['Inner Light Studio', 'Breathe Bliss', 'Serene Space']"
2,claude-3-sonnet-20240229,A 35-year-old artist exploring yoga for creative inspiration and community involvement,Zen Haven,"['Soul Sanctuary', 'Zen Haven', 'Blissful Balance']"
3,claude-3-sonnet-20240229,A 45-year-old mother of three looking to improve flexibility and find personal time,Breathe Bliss,"['Inner Light Studio', 'Breathe Bliss', 'Serene Space']"
4,claude-3-sonnet-20240229,A 60-year-old retired teacher using yoga to maintain health and social connections,Zen Haven,"['Inner Light Studio', 'Breathe Bliss', 'Serene Space']"
5,gpt-4o,A 22-year-old college athlete incorporating yoga for enhanced performance and injury prevention,Zen Flow,"['Soul Sanctuary', 'Breathe Bliss', 'Mindful Movements']"
6,gpt-4o,A 28-year-old tech professional seeking stress relief and mindfulness after long work hours,Zen Flow,"['Soul Sanctuary', 'Zen Flow', 'Breathe Bliss']"
7,gpt-4o,A 35-year-old artist exploring yoga for creative inspiration and community involvement,Inner Light Studio,"['Soul Sanctuary', 'Inner Light Studio', 'Breathe Bliss']"
8,gpt-4o,A 45-year-old mother of three looking to improve flexibility and find personal time,Zen Flow,"['Soul Sanctuary', 'Breathe Bliss', 'Blissful Balance']"
9,gpt-4o,A 60-year-old retired teacher using yoga to maintain health and social connections,Zen Haven,"['Zen Flow', 'Breathe Bliss', 'Serenity Studio']"


We could also solicit feedback on individual names.
This can be done by creating a "scenario" of each question for each name:

In [22]:
from edsl import QuestionLinearScale, QuestionFreeText, Survey, ScenarioList

In [23]:
scenarios = ScenarioList.from_list("studio_name", studio_names)
scenarios.table()

,studio_name
0,Soul Sanctuary
1,Zen Flow
2,Inner Light Studio
3,Karma Konnection
4,Namaste Nook
5,Breathe Bliss
6,Zen Haven
7,Soul Stretch
8,Harmony Hive
9,Harmony Haven


In [24]:
q_memorable = QuestionLinearScale(
    question_name = "memorable",
    question_text = "How memorable is this yoga studio name: {{ studio_name }}",
    question_options = [1,2,3,4,5],
    option_labels = {1:"Not at all memorable", 5:"Very memorable"}
)

In [25]:
q_criticism = QuestionFreeText(
    question_name = "criticism",
    question_text = "What are some criticisms people might have about this yoga studio name: {{ studio_name }}"
)

In [26]:
survey = Survey([q_memorable, q_criticism])

In [27]:
results = survey.by(scenarios).by(agents).by(models).run()

Job UUID,15e66de2-ef18-4dbd-bae0-6a3670c248f3
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/15e66de2-ef18-4dbd-bae0-6a3670c248f3
Exceptions Report URL,None
Results UUID,0af31e72-96a9-4bba-aad7-ae102ccdb338
Results URL,https://www.expectedparrot.com/content/0af31e72-96a9-4bba-aad7-ae102ccdb338


In [28]:
(
    results
    .sort_by("model", "persona")
    .select("model", "persona", "studio_name", "memorable", "criticism")
)

,model.model,agent.persona,scenario.studio_name,answer.memorable,answer.criticism
0,claude-3-sonnet-20240229,A 22-year-old college athlete incorporating yoga for enhanced performance and injury prevention,Soul Sanctuary,5,"As a 22-year-old college athlete incorporating yoga for enhanced performance and injury prevention, here are some potential criticisms I could envision about the name ""Soul Sanctuary"" for a yoga studio: 1. Lack of relatability for athletes: The name has a very spiritual/new age connotation that may not resonate as much with athletes who are primarily interested in the physical benefits of yoga for performance and injury prevention. A name more focused on strength, flexibility, or athletic performance could be preferred. 2. Religious overtones: The word ""soul"" could be interpreted as having religious undertones that some may find off-putting or exclusionary, especially for those not aligning with certain spiritual beliefs. 3. Overused/cliché: Words like ""soul"" and ""sanctuary"" are quite common in the yoga/wellness space, so the name may come across as a bit cliché or unoriginal. 4. Mismatch with athletic vibe: College athletes may prefer a name that sounds more energetic, powerful and aligned with an athletic/training mentality versus the more relaxing, meditative vibe of ""Soul Sanctuary."" 5. Confusion on offerings: The name implies a focus on the spiritual/meditative aspects of yoga, which could misrepresent the more physical, athletic style of yoga being offered. However, it's ultimately a subjective matter of personal preferences. The name could also resonate well with many athletes seeking that mind-body connection through yoga. Proper branding and messaging can help align expectations."
1,claude-3-sonnet-20240229,A 22-year-old college athlete incorporating yoga for enhanced performance and injury prevention,Zen Flow,4,"As a 22-year-old college athlete incorporating yoga for enhanced performance and injury prevention, here are some potential criticisms I could see with the yoga studio name ""Zen Flow"": 1. Overused words: The words ""zen"" and ""flow"" are quite commonly used in the yoga/wellness space. Some may view it as an unoriginal or cliched name that doesn't stand out. 2. Lack of specificity: The name doesn't really convey anything specific about the type of yoga taught, the studio's philosophy, or what makes it unique. It's quite generic. 3. Cultural appropriation concerns: The word ""zen"" has roots in Buddhism. Some may take issue with a Western business using the term without proper cultural context or representation. 4. Disconnect for athletes: As an athlete, the name ""Zen Flow"" doesn't immediately make me think of a yoga practice geared towards enhancing sports performance or preventing injuries. It sounds more spiritual/meditative. That said, the name does have a calm, peaceful vibe that some yoga practitioners may be drawn to. But for an athlete like myself, I may prefer a name that highlights the physical and practical benefits I'm seeking from yoga. Overall, a more distinctive and descriptive name could be an improvement."
2,claude-3-sonnet-20240229,A 22-year-old college athlete incorporating yoga for enhanced performance and injury prevention,Inner Light Studio,4,"As a 22-year-old college athlete incorporating yoga for enhanced performance and injury prevention, here are some potential criticisms I could see with the name ""Inner Light Studio"" for a yoga studio: 1. Cliché or overused: The phrase ""inner light"" is fairly common in the yoga/spiritual realm, so some may view it as an unoriginal or clichéd name choice. 2. Too spiritual/religious: Depending on the studio's focus, some people may interpret ""inner light"" as having overly spiritual or religious connotations that don't align with their interests if they just want a physical yoga practice. 3. Unclear meaning: For those unfamiliar with yoga philosophy, the meaning behind ""inner light"" may not be immediately clear o

## Posting to the Coop
The [Coop](https://www.expectedparrot.com/explore) is a platform for creating, storing and sharing LLM-based research.
It is fully integrated with EDSL and accessible from your workspace or Coop account page.
Learn more about [creating an account](https://www.expectedparrot.com/login) and [using the Coop](https://docs.expectedparrot.com/en/latest/coop.html).

Here we demonstrate how to post this notebook to share with others (visibility can be *public*, *private* or *unlisted* by default):

In [29]:
from edsl import Notebook

In [30]:
n = Notebook(path = "yoga_studio_name_survey.ipynb")

In [31]:
info = n.push(description = "Feedback on names for a yoga studio", visibility = "public")
info

{'description': 'Feedback on names for a yoga studio',
 'object_type': 'notebook',
 'url': 'https://www.expectedparrot.com/content/b8a80348-705b-4aa5-b181-9bea22c00666',
 'uuid': 'b8a80348-705b-4aa5-b181-9bea22c00666',
 'version': '0.1.45.dev1',
 'visibility': 'public'}

Update an object at Coop:

In [32]:
n = Notebook(path = "yoga_studio_name_survey.ipynb") # resave

In [33]:
n.patch(uuid = info["uuid"], value = n)

{'status': 'success'}